In [1]:
import asyncio
from lionagi import CallDecorator as cd

the decorator works with both sync and async functions

`@cd.cache`

keep function result in memory

In [2]:
import time

@cd.cache
def square_data(x):
    time.sleep(1)
    return x**2

result, elapse = await li.tcall(input_=10_000, func=square_data, sleep=0, include_timing=True)
print(f"result: {result:_}, elapse: {elapse:.03f} seconds")

# the second time when the function runs with same parameter, it will return the result directly
result, elapse = await li.tcall(input_=10_000, func=square_data, sleep=0, include_timing=True)
print(f"result: {result:_}, elapse: {elapse:.03f} seconds")

result: 100_000_000, elapse: 1.002 seconds
result: 100_000_000, elapse: 0.000 seconds


In [3]:
@cd.cache
async def asquare_data(x):
    await asyncio.sleep(1)
    return x * x

result, elapse = await li.tcall(input_=10_000, func=asquare_data, sleep=0, include_timing=True)
print(f"result: {result:_}, elapse: {elapse:.03f} seconds")

result, elapse = await li.tcall(input_=10_000, func=asquare_data, sleep=0, include_timing=True)
print(f"result: {result:_}, elapse: {elapse:.03f} seconds")

result: 100_000_000, elapse: 1.002 seconds
result: 100_000_000, elapse: 0.000 seconds


`@cd.filter`

In [4]:
@cd.filter(predicate=lambda x: x<10)
def square_data(x):
    return [0, x**2]
    
square_data(5)

[0]

`@cd.throttle` 

In [5]:
# sync
# this block should take 5 seconds to run
@cd.throttle(1)
def square_data(x):
    return(x**2)

print(li.lcall(range(5), square_data))

[0, 1, 4, 9, 16]


In [6]:
# async
# this block should take one second to run
@cd.throttle(1)
async def square_data(x):
    return(x**2)

print(await li.alcall(range(5), await square_data))

[0, 1, 4, 9, 16]


`@cd.pre_post_process`

In [7]:
# sync
@cd.pre_post_process(preprocess=lambda x: x+1, postprocess=lambda x: x**2)
def get_5(x):
    return x

print(get_5(5))

# async
@cd.pre_post_process(preprocess=lambda x: x-1, postprocess=lambda x: x**3)
async def get_5(x):
    return x

print(await get_5(5))

36
64


`@cd.timeout`

In [8]:
@cd.timeout(1)
async def timeout_function():
    await asyncio.sleep(2)
    return "Completed"

try:
    result = await timeout_function()
    print(result)
except asyncio.TimeoutError:
    print("Function call timed out")

Function call timed out


`@cd.retry`

In [9]:
@cd.retry(retries=3, initial_delay=1)
async def retry_function(attempt):
    if attempt < 3:
        raise ValueError("Simulated error")
    return "Success"

try:
    result = await retry_function(1)
    print(result)
except ValueError as e:
    print(f"Function call failed after retries: {e}")

Function call failed after retries: Simulated error


`@cd.default`

In [10]:
@cd.default(default_value="Default Result")
async def default_function(may_fail):
    if may_fail:
        raise ValueError("Simulated error")
    return "Success"

result = await default_function(True)
print(result)

Default Result


`@cd.compose`

In [11]:
f1 = lambda x: x+1
f2 = lambda x: x*2

@cd.compose(f1, f2)
def increment_and_double(x):
    return x

result = increment_and_double(3)
print(result)

8


`@cd.max_concurrency`

In [12]:
# this should take 3 seconds to run
@cd.max_concurrency(limit=2)
async def limited_concurrency_function(x):
    await asyncio.sleep(1)
    return x * 2

async def run_max_concurrency_example():
    tasks = [limited_concurrency_function(i) for i in range(5)]
    results = await asyncio.gather(*tasks)
    print(results)

await run_max_concurrency_example()

[0, 2, 4, 6, 8]


`@cd.memorize`

In [13]:
# you can specify how many function calls to remember
@cd.memorize(maxsize=10)
def memorized_function(x):
    time.sleep(1)
    return x * x

start_time = time.time()
result1 = memorized_function(2)
duration1 = time.time() - start_time
print(f"First call result: {result1}, Duration: {duration1:.03f}")

start_time = time.time()
result2 = memorized_function(2)
duration2 = time.time() - start_time
print(f"Second call result: {result2}, Duration: {duration2:.03f}")

First call result: 4, Duration: 1.001
Second call result: 4, Duration: 0.000


`@cd.map`

In [14]:
@cd.map(lambda x: x * x)
def square_numbers(numbers):
    return numbers

numbers = [1, 2, 3, 4, 5]
squared_numbers = square_numbers(numbers)
print(squared_numbers)

[1, 4, 9, 16, 25]


`@cd.reduce`

In [15]:
@cd.reduce(lambda x, y: x + y, 0)
def sum_numbers(numbers):
    return numbers

numbers = [1, 2, 3, 4, 5]
total = sum_numbers(numbers)
print(total)

15
